# Analyse fréquentielle intégratrice

## Activité de transfert - 21 juillet 2022

préparée par :
- Duy Anh Alexandre
- Gabriel Gobeil
- Jonathan Jalbert

## Application 2 : Analyse fréquentielle des pseudo-observations

Dans cette application, vous reprendrez les grandes étapes du calepin 2-PseudoObservationsFrequencyAnalysis.ipynb pour le tronçon de la rivière Famine : SLSO00160

Vous n'avez qu'à compléter les cellules où il est spécificié **#TODO**.

---
# 1. Chargement des librairies et des fonctions utiles <a name="preparation"></a>

In [ ]:
# Chargement des librairies
using CSV, DataFrames
using Distributions, Extremes, ErrorsInVariablesExtremes, Mamba
using Gadfly

---
# 2. Chargement des données <a name="data"></a>

## 2.1 Chargement des pseudo-observations <a name="data_loading"></a>
 

#### Chargement des données

On charge les données directement à partir du fichier NetCDF avec la fonction `load_discharge_distribution(filename)` où `filename` est le nom du fichier contant les données. Les données sont chargées dans un vecteur de [Pseudodata](https://juliaextremes.github.io/ErrorsInVariablesExtremes.jl/dev/functions/#ErrorsInVariablesExtremes.Pseudodata), une structure de donnée qui sert à encapsuler les distributions de probabilité pour chacune des données non observées.

In [ ]:
filename = "../0-Data/0-PseudoObs/A2020_Analyse_Historique_QMA_SLSO00160.nc"
pdata = load_discharge_distribution(filename)

## 2.2 Analyse exploratoire <a name="exploration"></a>

Analyse exploratoire pour visualiser les pseudo-observations.

#### Conversion en DataFrame

La fonction `convert` permet de convertir le vecteur de `Pseudodata` en `DataFrame` afin de faciliter la manipulation des données avec les méthodes adaptées aux DataFrame.

In [ ]:
df = convert(DataFrame, pdata)
first(df,5)

#### Calcul de la moyenne ainsi que de l'intervalle de confiance à 1-α pour chacune des configurations

À partir des distributions, il est possible de calculer facilement les quantités d'intérêt pour tracer la moyenne et l'intervalle de confiance pour chacune des configuration.

In [ ]:
# 1-α correspond au niveau de l'intervalle de confiance
α = 0.5

# Calcul de la médiane
df[:, :y] = median.(df.Distribution)

# Calcul du quantile inférieur de l'intervalle de confiance
df[:, :ymin] = quantile.(df.Distribution, α/2)

# Calcul du quantile supérieur de l'intervalle de confiance
df[:, :ymax] = quantile.(df.Distribution, 1-α/2)

first(df, 5)

#### Affichage de la médiane et de l'intervalle de confiance pour toutes les configurations

In [ ]:
set_default_plot_size(12cm, 8cm)
plot(df, x=:Year, y=:y, color=:Configuration, Geom.line,
    ymin=:ymin, ymax=:ymax, Geom.ribbon,
    Guide.ylabel("Discharge [m³/s]"))

#### Affichage de la médiane et de l'intervalle de confiance pour une configuration

In [ ]:
config = "MG24HQ"

df2 = filter(row -> row.Configuration ==config, df)

set_default_plot_size(12cm, 8cm)
plot(df2, x=:Year, y=:y, Geom.line, Geom.point,
    ymin=:ymin, ymax=:ymax, Geom.ribbon,
    Guide.ylabel("Discharge [m³/s]"),
    Guide.title(config))

---
# 3. Modélisation bayésienne des maxima annuels non observés <a name="modelisation"></a>


## 3.1 Modèle stationnaire <a name="mstat"></a>

### 3.1.1 Définition du modèle

On encapsule les données et la loi *a priori* dans la structure `PseudoMaximaModel` qui constitue le modèle statistique :

In [ ]:
model1 = PseudoMaximaModel(pdata, prior=[Flat(), Flat(), LocationScale(-.5, 1, Beta(6,9))])

### 3.1.2 Génération d'un échantillon de la loi *a posteriori* par MCMC et estimation des paramètres

La fonction `fitbayes` permet d'obtenir un échantillon de la loi *a posteriori* des paramèetres et des maxima annuels. 

La fonction prend en entrée un modèle de type `PseudoMaximaModel`.

In [ ]:
#TODO: Générer un échantillon aléatoire de la loi *a posteriori* du modèle "model1".


### 3.1.3 Affichage des estimations des maxima annuels

À partir des chaînes inférées, il est possible de calculer facilement les quantités d'intérêt pour tracer une estimation ponctuelle des maxima annuels et son intervalle de crédibilité.

In [ ]:
#TODO: Calculez l'estimation des maxima annuels


In [ ]:
#TODO: Calculez les intervalles de crédibilité des maxima annuels


In [ ]:
#TODO: Tracez la séries des maxima estimés ainsi que l'intervalle de crédibilité


### 3.1.4 Adéquation de la loi GEV ajustée aux estimations des maxima annuels

Une fois les paramètres estimés, on procéde à une validation graphique en traçant les figures diagnostiques à l'aide de la fonction `diagnosticplots`.

#### Adéquation générale avec les résidus

In [ ]:
#TODO: Vérifiez l'adéquation du modèle à l'aide des figures diagnostiques


### 3.1.5 Estimation des quantiles

Les estimations des niveaux de retour associés à la période $T$ peuvent être obtenues en calculant les quantiles d'ordre $\big(1-\frac{1}{T}\big)$ à l'aide de la fonction `quantile`. 

Par exemple, ici on calcule les niveaux de retour associés à une période de retour de 20 ans et on affiche la densité pour les 1001 itérations.

In [ ]:
# Période de retour
T = 20

#TODO: Calculez les quantiles correspondant à cette période de retour

## 3.2 Modèle non stationnaire  <a name="mnstat1"></a>

Maintenant, on considère un modèle non stationnaire où le paramètre de localisation de la loi GEV est une fonction de la concentration de GES dans l'atmosphère.

### 3.2.1 Chargement des variables exlicatives

On doit d'abord charger puis construire la variable avec la fonction `buildVariables` de la librairie *Extremes.jl*.

In [ ]:
# Chargement de la concentration équivalente de CO2
co2data = CSV.read("../0-Data/0-RCP/RCPdata.csv", DataFrame);
filter!(row->row.Year in pdata[1].year, co2data)

# Standardisation de la variable explicative
co2data[:, :RCP85std] = (co2data.RCP85 .- mean(co2data.RCP85)) ./ std(co2data.RCP85)

# Encapsulation dela variable explicatice pour une utilisation efficace
locationcov = Extremes.buildVariables(co2data, [:RCP85std])

### 3.2.2 Définition du modèle

Comme à la section 3.1.1, on encapsule les données et la loi *a priori* dans la strucutre `PseudoMaximaModel`. Cette fois-ci, on spécifie la variable explicative sur le paramètre de localisation avec l'argument `locationcov` pour définir le modèle non stationnaire. La loi *a priori* a maintenant une dimension supplémentaire mais elle reste informative que pour le paramètre de forme.

In [ ]:
model2 = PseudoMaximaModel(pdata, locationcov = locationcov, 
    prior=[Flat(), Flat(), Flat(), LocationScale(-.5, 1, Beta(6,9))])

### 3.2.3 Génération d'un échantillon de la loi *a posteriori* par MCMC et estimation des paramètres

Comme pour le modèle stationnaire, la fonction `fitbayes` permet de générer un échantillon de la loi *a posteriori*  du modèle bayésien non stationnaire défini précédement. 

In [ ]:
#TODO : Générer un échantillon de la loi a posteriori des paramètres et des maxima


### 3.2.4 Affichage des estimations des maxima annuels

In [ ]:
#TODO: Calculez l'estimation des maxima annuels


In [ ]:
#TODO: Calculez les intervalles de crédibilité des maxima annuels


In [ ]:
#TODO: Tracez la séries des maxima estimés ainsi que l'intervalle de crédibilité


### 3.2.5 Adéquation de la loi GEV ajustée aux estimations des maxima annuels

#### Adéquation générale avec les résidus

In [ ]:
#TODO: Vérifiez l'adéquation du modèle à l'aide des figures diagnostiques


### 3.2.6 Estimation des niveaux de retour effectifs

Les estimations des niveaux de retour effectifs associés à la période $T$ **à chaque année** peuvent être obtenues en calculant les quantiles d'ordre $\big(1-\frac{1}{T}\big)$ à l'aide de la fonction `quantile`. 

In [ ]:
# Période de retour
T = 20

#TODO: Calculez les quantiles correspondants 
# Difficile car les quantiles changent à chaque année
# La façon que l'on propose de faire dans l'exemple et de traiter l'année comme une catégorie en Julia.
# Il y a d'autres approches possibles.

---
# 4. Sélection de modèle <a name="selection"></a>

La sélection de modèle se fait avec le critère d'information de déviance (DIC). La fonction `dic()` appliquée au modèle ajusté permet de calculer ce critère.

In [ ]:
#TODO: Calculez le DIC pour le modèle stationnaire et le modèle non stationnaire.


Un modèle est considéré meilleur qu’un autre lorsque la valeur du DIC correspondant est plus faible. On cherche  à déterminer le modèle avec la plus petite valeur de DIC avec la fonction `argmin(res)` qui retourne l'indice du plus petit élément de `res`.

In [ ]:
#TODO: Sélectionnez le meilleur modèle.


---
# 5. Enregistrement du modèle <a name="enregistrement"></a>

Pour pouvoir utiliser les résultats de l'ajustement du modèle dans les parties suivantes de l'activité sans avoir à tout recalculer, il est utile d'enregistrer les résultats. La librairie Julia de base `Serialization` permet de sérialiser la structure `PseudoMaximaEVA` et de l'exporter sous forme de fichier .txt qui pourra être désérialiser dans les parties suivantes.

In [ ]:
using Serialization

io = open("2-Results/Famine_pseudoObs.txt", "w");

#TODO: Remplacez fm1 par le nom de votre meilleur modèle dans la commande suivante et exécutez-la.
serialize(io, fm1);

close(io);